<a href="https://colab.research.google.com/github/angulorojasmariaclaudia-coder/TRABAJO-FINAL-TECNICAS-DE-CIENCIAS-DE-DATOS/blob/main/Configuraci%C3%B3n_de_C%C3%B3digo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **¡¡IMPORTANTE!! Para enterder el DATASET**

🎯 **OBJETIVO REAL**

Clasificar qué tan mal contaminada está cada Comunidad Autónoma (CCAA)
usando solo los contaminantes, no enfermedades.

\

🎯 Para clasificar la contaminación de una CCAA, necesitamos **solo los contaminantes**:
* Air Pollution Average [ug/m3]
* Air Pollution Population Weighted Average[ug/m3]
* Air Pollutant (pivotado a PM25/NO2/O3)
* Year
* Affected Population
* Populated Area [km2]
* CCAA: Comunidad Autónoma

\

🧠 El conjunto de datos contiene registros “**Mortality**” con las enfermedades como 'Diabetes' o cánceres. **NO** debemos usar `Mortality`, `Outcome`, `Attributable deaths (AD)`, `Value` (valor estimado de muertes en base a la OMS), ni `Value for 100k…`. Porque es información derivada de la contaminación.

La parte de “enfermedades” solo sirve para:

* ✔️ tu análisis exploratorio
* ✔️ entender impacto de salud
* ❌ NO para el modelo


\

**ELIMINAREMOS LAS ENFERMEDADES DEL DATASET**

\

**IMPORTANTE TAMBIÉN:**
* Años considerados: 2018 a 2022.
* Edad de la población >= 25 (población expuesta a la contaminación).
* Contaminantes: NO2, PM2.5	y O3.
* Población: Urban Centres (grid).


# Paquetes

In [1]:
import pandas as pd
from google.colab import files

# Conjunto de datos

In [2]:
uploaded = files.upload()
df = pd.read_csv('causas3.csv')

Saving causas3.csv to causas3 (3).csv


In [3]:
df.shape # numero de filas y columnas

(25440, 23)

In [ ]:
df['Year'].value_counts() # años de estudio

In [ ]:
df[df["Year"] == 2020] # ejemplo de ciudad (todas tienen las mismas caracteríticas)

In [4]:
df.head(10) # dataset original

,City Boundary Specification (LAU/grid),Country Or Territory,City Or Territory,City Code,Year,Air Pollutant,Data Aggregation Id,Scenario,Category,Outcome,...,Affected Population,Populated Area [km2],Air Pollution Average [ug/m3],Air Pollution Population Weighted Average [ug/m3],Value,Value - lower CI,Value - upper CI,Value for 100k Of Affected Population,Value for 100k Of Affected Population - lower CI,Value for 100k Of Affected Population - upper CI
0,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2021,NO2,P1Y,Baseline from WHO 2021 AQG,Mortality,Diabetes Mellitus,...,41314,9.0,13.0,13.8,8,4,11,19,10,28
1,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2021,O3,SOMO35,Baseline from WHO 2021 AQG,Mortality,All causes,...,54686,9.0,4742.1,4735.3,2,2,3,5,4,6
2,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2021,PM2.5,P1Y,Baseline from WHO 2021 AQG,Morbidity,Stroke,...,41314,9.0,7.3,7.4,3,0,7,9,0,18
3,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2021,PM2.5,P1Y,Baseline from WHO 2021 AQG,Mortality,Diabetes Mellitus,...,41314,9.0,7.3,7.4,13,6,19,31,16,46
4,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2021,PM2.5,P1Y,Baseline from WHO 2021 AQG,Mortality,Stroke,...,41314,9.0,7.3,7.4,1,0,2,3,1,6
5,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2021,PM2.5,P1Y,Baseline from WHO 2021 AQG,Total burden of disease,Lung cancer,...,41314,9.0,7.3,7.4,19,7,31,47,17,76
6,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2022,NO2,P1Y,Baseline from WHO 2021 AQG,Mortality,Diabetes Mellitus,...,41405,9.0,13.3,14.4,9,4,13,22,11,32
7,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2022,NO2,P1Y,Baseline from WHO 2021 AQG,Total burden of disease,Asthma,...,47078,9.0,13.3,14.4,3,0,6,7,1,14
8,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2022,NO2,P1Y,Baseline from WHO 2021 AQG,Total burden of disease,Diabetes Mellitus,...,41405,9.0,13.3,14.4,18,9,26,43,23,65
9,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2022,NO2,P1Y,Baseline from WHO 2021 AQG,Total burden of disease,Stroke,...,41405,9.0,13.3,14.4,9,2,17,23,6,41


# Preparación del conjunto de datos

### Añadir variables CCAA

Vamos a ver las ciudades que reúne nuestro conjunto de datos.

In [ ]:
df["City Or Territory"].unique()

Vemos que hay dos casillas, Santa Mónica y All Urban Centres in a Country, que no son ciudades de España. Por lo que las quitaremos de nuestro conjunto de datos.

In [ ]:
df = df[df["City Or Territory"] != "All Urban Centres in a Country"]
df = df[df["City Or Territory"] != "Santa Monica"]

A continuación, vamos a añadir una columna que indique la Comunidad Autónoma según la ciudad registrada en nuestro conjunto de datos.

In [ ]:
mapa_ccaa = {
    'Collado Villalba': 'Comunidad de Madrid', 'Terrassa': 'Cataluña', 'Toledo': 'Castilla-La Mancha',
    'Santiago de Compostela': 'Galicia','Sevilla': 'Andalucía','Vitoria/Gasteiz': 'País Vasco',
    'Vigo': 'Galicia','Viladecans': 'Cataluña','Fuenlabrada': 'Comunidad de Madrid',
    'Gandia': 'Comunidad Valenciana','Torremolinos': 'Andalucía','El Prat de Llobregat': 'Cataluña',
    'Valencia': 'Comunidad Valenciana','Valladolid': 'Castilla y León','Tomares': 'Andalucía',
    'Tarragona': 'Cataluña','Torrevieja': 'Comunidad Valenciana','Eivissa': 'Islas Baleares',
    'el Masnou': 'Cataluña','Torrelavega': 'Cantabria','Cuenca': 'Castilla-La Mancha',
    'Ferrol': 'Galicia','Guadalajara': 'Castilla-La Mancha','Huelva': 'Andalucía',
    'Igualada': 'Cataluña','Vilanova i la Geltrú': 'Cataluña','Granada': 'Andalucía',
    'Zamora': 'Castilla y León','Granollers': 'Cataluña','Torrejón de Ardoz': 'Comunidad de Madrid',
    'Gijón': 'Asturias','Santander': 'Cantabria','Córdoba': 'Andalucía','Talavera de la Reina': 'Castilla-La Mancha',
    'León': 'Castilla y León','Linares': 'Andalucía','Línea de la Concepción, La': 'Andalucía',
    'Lleida': 'Cataluña','Logroño': 'La Rioja',"El Port de Sagunt / Canet d'En Berenguer": 'Comunidad Valenciana',
    'Dos Hermanas': 'Andalucía','Irun': 'País Vasco','Madrid': 'Comunidad de Madrid',
    'Majadahonda': 'Comunidad de Madrid','Fuengirola': 'Andalucía','Valdemoro': 'Comunidad de Madrid',
    'Alcalá de Guadaíra': 'Andalucía','Coslada': 'Comunidad de Madrid','Oviedo': 'Asturias',
    'Algeciras': 'Andalucía','San Sebastián/Donostia': 'País Vasco','Sanlúcar de Barrameda': 'Andalucía',
    'Almería': 'Andalucía','Ávila': 'Castilla y León','Palma de Mallorca': 'Islas Baleares','Pamplona/Iruña': 'Navarra','Avilés': 'Asturias',
    'Pontevedra': 'Galicia','Pozuelo de Alarcón': 'Comunidad de Madrid','Alicante/Alacant': 'Comunidad Valenciana',
    'Palencia': 'Castilla y León','Zaragoza': 'Aragón','Girona': 'Cataluña','Sabadell': 'Cataluña',
    'Salamanca': 'Castilla y León','San Fernando': 'Andalucía','Manresa': 'Cataluña','Marbella': 'Andalucía','Parla': 'Comunidad de Madrid',
    'Ponferrada': 'Castilla y León', 'Melilla': 'Ciudad Autónoma de Melilla','A Coruña': 'Galicia','Albacete': 'Castilla-La Mancha',
    'Mérida': 'Extremadura','Murcia': 'Región de Murcia','Alcalá de Henares': 'Comunidad de Madrid',
    'Ourense': 'Galicia','Alcoi/Alcoy': 'Comunidad Valenciana','Sant Cugat del Vallès/Rubí': 'Cataluña',
    'Sant Boi de Llobregat': 'Cataluña',
    'Jaén': 'Andalucía',
    'Jerez de la Frontera': 'Andalucía',
    'Lorca': 'Región de Murcia',
    'Lugo': 'Galicia',
    'Benidorm': 'Comunidad Valenciana',
    'Bilbao': 'País Vasco',
    'Puerto de Santa María, El': 'Andalucía','Reus': 'Cataluña','Ciudad Real': 'Castilla-La Mancha',
    'Badajoz': 'Extremadura','Burgos': 'Castilla y León','Cáceres': 'Extremadura',
    'Barcelona': 'Cataluña','Málaga': 'Andalucía','Elche/Elx': 'Comunidad Valenciana',
    'Elda': 'Comunidad Valenciana','Mataró': 'Cataluña','Cádiz': 'Andalucía',
    'Cartagena': 'Región de Murcia','Castelldefels': 'Cataluña','Mollet del Vallès': 'Cataluña',
    'Castellón de la Plana/Castelló de la Plana': 'Comunidad Valenciana','Ceuta': 'Ciudad Autónoma de Ceuta',
    'Chiclana de la Frontera': 'Andalucía'
}

df["CCAA"] = df["City Or Territory"].map(mapa_ccaa)

In [ ]:
df["CCAA"].unique() # visualizamos los resultados

### Filtrado filas y columnas DataFrame Original

Filtramos las filas por:
* Ciudades que son de la `Urban Centres (grid)`
* Los que se atribuyen a las muertes `Health Indicator = Attributable deaths (AD)`.
* Edades: `>= 25 years of age"` y `>= 30 years of age`

Por columnas nos quedaremos con `CCAA`,`City Or Territory`, `Air Pollutant`, `Year`, `Air Pollution Average [ug/m3]`, `Air Pollution Population Weighted Average [ug/m3]`, `Affected  Population`, `Populated Area [km2]`.

In [ ]:
df = df[
    (df["City Boundary Specification (LAU/grid)"] == "Urban Centres (grid)") &
    (df["Description Of Age Group"].isin([">= 25 years of age"])) &
    (df["Health Indicator"] == "Attributable deaths (AD)")
]

In [ ]:
df = df.sort_values( # ordenamos el dataset
    by=["CCAA", "City Or Territory", "Year", "Air Pollutant"]
).reset_index(drop=True)

Vamos a seleccionar columnas de interés y eliminar duplicados.

In [ ]:
columnas = ['CCAA','City Or Territory','Air Pollutant','Year',
            'Air Pollution Average [ug/m3]',
            'Air Pollution Population Weighted Average [ug/m3]',
            'Affected  Population', 'Populated Area [km2]']

df = df[columnas].copy()

In [ ]:
df = df.drop_duplicates()

Comprobamos que todas las ciudades reúnen las mismas características de estudio y eliminamos aquellos registros que añadan información irrelevante por ser un dato duplicado.

In [ ]:
pd.set_option('display.max_rows', None)
df["City Or Territory"].value_counts()


In [ ]:
indices_eliminar = [642, 639, 2555, 2560, 2440]
df = df.drop(index=indices_eliminar)

# **DATOS "LIMPIOS"**

El dataset tiene 1568 instancias y 8 variables de estudio.

In [ ]:
df.shape

In [ ]:
df.head(4)

In [ ]:
df[df["City Or Territory"] == "Eivissa"] # ejemplo de ciudad (todas tienen las mismas caracteríticas)
